In [ ]:
import requests
import tabula
import pandas as pd

# Download of Module Handbooks

In [ ]:
pdf_url_bachelor_business_management_and_economics = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-184-H-2008.pdf"
pdf_url_bachelor_information_systems = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-277-H-2021.pdf"
pdf_url_bachelor_economathematics = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-276-H-2015.pdf"
pdf_url_master_information_systems = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-j10-H-2019.pdf"
pdf_url_master_management ="https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-i45-H-2018.pdf"
pdf_url_master_international_economic_policy = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-h92-H-2018.pdf"

response = requests.get(pdf_url_bachelor_business_management_and_economics)
with open("module_handbooks/bachelor_business_management_and_economics.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded bachelor_business_management_and_economics.pdf")

response = requests.get(pdf_url_bachelor_information_systems)
with open("module_handbooks/bachelor_information_systems.pdf", "wb") as f:
    f.write(response.content)

print("Downloaded bachelor_information_systems.pdf")
response = requests.get(pdf_url_bachelor_economathematics)
with open("module_handbooks/bachelor_economathematics.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded bachelor_economathematics.pdf")

response = requests.get(pdf_url_master_information_systems)
with open("module_handbooks/master_information_systems.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_information_systems.pdf")

response = requests.get(pdf_url_master_management)
with open("module_handbooks/master_management.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_management.pdf")

response = requests.get(pdf_url_master_international_economic_policy)
with open("module_handbooks/master_international_economic_policy.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_international_economic_policy.pdf")

# Ansatz über TABULA --> Verforfen 
Reading Tables from Module Book for master_information_systems

In [ ]:
import pandas as pd
import tabula
import re

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:


tables_master_is = tabula.read_pdf("module_handbooks/master_information_systems.pdf", pages='10-30', multiple_tables=True)
print(len(tables_master_is))
print(tables_master_is)

# PDF Miner Ansatz wie Felix -> über Regex

In [ ]:
import io
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFPageInterpreter, PDFResourceManager
from pdfminer.pdfpage import PDFPage


def pdf_to_text(file_path, n=0):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    laparams = LAParams()
    converter = TextConverter(
        resource_manager,
        fake_file_handle,
        codec="utf-8",
        laparams=laparams,
    )
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(file_path, "rb") as fh:
        for i, page in enumerate(PDFPage.get_pages(fh, caching=True, check_extractable=True)):
            if i < n:
                continue
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    # close open handles
    converter.close()
    fake_file_handle.close()

    if text:
        return remove_header_footer(text)
    else:
        return ""


def remove_header_footer(text):
    # Split the text into lines
    lines = text.split("\n")

    # Remove lines that match the header pattern
    lines = [line for line in lines if not line.startswith("JMU Würzburg")]

    # Remove lines that match the footer pattern
    lines = [line for line in lines if not line.startswith("page")]
    lines = [line for line in lines if not re.match(r".*Master.* with 1 major Information Systems.*", line)]
    lines = [line for line in lines if not re.match(r".*Master.* Information Systems - 2019.*", line)]
    lines = [line for line in lines if not re.match(r".*Module Catalogue for the Subject.*", line)]
    lines = [line for line in lines if not re.match(r".*Information Systems.*", line)]
    lines = [line for line in lines if not re.match(r".*Master’s with 1 major, 120 ECTS credits.*", line)]


    # Join the remaining lines back into a single string
    clean_text = "\n".join(lines)

    return clean_text


In [ ]:
text = pdf_to_text("module_handbooks/master_information_systems.pdf", n=9)

In [ ]:
text_no_footer_header = remove_header_footer(text)
print(text_no_footer_header)

with open("module_handbooks/master_information_systems_processed.txt", "w") as f:
    f.write(text_no_footer_header)

In [ ]:
tables = re.split(r"(?=Module title)", text_no_footer_header)[1:]
print(len(tables))
print("---------------TABLE 0--------------------------")
print(tables[0])
# store to table.txt file 
with open("module_handbooks/master_information_systems_table_0.txt", "w") as f:
    f.write(tables[0])

print("---------------TABLE 1--------------------------")
print(tables[1])

In [ ]:
def extract_table_values(table_string):
    # Define regex patterns
    patterns = {
        "Module Title": r"Module title\s+(.*?)\s+Abbreviation",
        "Abbreviation": r"Abbreviation\s+(.*?)\s+Module Coordinator",
        "Module Coordinator": r"Module coordinator\s+(.*?)\s+Module offered by",
        "Module Offered By": r"Module offered by\s+(.*?)\s+ECTS Method of grading",
        "ECTS": r"ECTS Method of grading\s+(.*?)\s+Method of grading",
        "Method of Grading": r"Method of grading\s+(.*?)\s+Only after succ\. compl\. of module\(s\)",
        "Completion of Modules": r"Only after succ\. compl\. of module\(s\)\s+(.*?)\s+Duration",
        "Duration": r"Duration\s+(.*?)\s+Module level",
        "Module Level": r"Module level\s+(.*?)\s+Other prerequisites",
        "Other Prerequisites": r"Other prerequisites\s+(.*?)\s+Contents",
        "Contents": r"Contents\s+(.*?)\s+Intended learning outcomes",
        "Intended Learning Outcomes": r"Intended learning outcomes\s+(.*?)\s+Courses \(type, number of weekly contact hours, language — if other than German\)",
        "Courses": r"Courses \(type, number of weekly contact hours, language — if other than German\)\s+(.*?)\s+Method of assessment \(type, scope, language — if other than German, examination offered — if not every semester, information on whether module is creditable for bonus\)",
        "Method of Assessment": r"Method of assessment \(type, scope, language — if other than German, examination offered — if not every semester, information on whether module is creditable for bonus\)\s+(.*?)\s+Allocation of places",
        "Allocation of Places": r"Allocation of places\s+(.*?)\s+Additional information",
        "Additional Information": r"Additional information\s+(.*?)\s+Workload",
        "Workload": r"Workload\s+(.*?)\s+Teaching cycle",
        "Teaching Cycle": r"Teaching cycle\s+(.*?)\s+Referred to in LPO I",
        "Referred to in LPO I": r"Referred to in LPO I\s+(.*?)\s+Page",
        # add a Module appears in pattern 
        "Module appears in": r"Module appears in\n(.+)$"

    }

    extracted_values = {}

    # Extract values using regex
    for key, pattern in patterns.items():
        match = re.search(pattern, table_string, re.DOTALL)
        if match:
            extracted_values[key] = match.group(1).strip()

    return extracted_values


In [ ]:
values = extract_table_values(tables[0])
print(values)

# Print the extracted values
for key, value in values.items():
    print(key + ": " + value)

In [ ]:
for table in tables:
        values = extract_table_values(table)
        print("Done with extracting values for table")
        
        # Store the values in the matching column of the module_handbook_df
        row = {}
        for column in df.columns:
            if column in values:
                row[column] = values[column]
            else:
                row[column] = ""
        df = df.append(row, ignore_index=True)

# Create final Dataframe with these columns

In [ ]:
module_handbook_columns = [
    "module title",
    "abbreviation",
    "module coordinator",
    "module offered by",
    "ects",
    "method of grading",
    "Only after successfull completion of module(s)",
    "Duration",
    "Module level",
    "Other prerequisites",
    "Contents",
    "intended learning outcomes",
    "courses",
    "method of assessment",
    "allocation of places",
    "additional information",
    "workload",
    "teaching cycle",
    "referred to in lpo I",
    "page",
]
module_handbook_df = pd.DataFrame(columns=module_handbook_columns)
module_handbook_df.to_csv(
    "module_handbooks/master_information_systems.csv", index=False
)

In [ ]:
def process_module_handbook(tables_text):
    module_handbook_columns = [
        "Module title",
        "Abbreviation",
        "Module coordinator",
        "Module offered by",
        "ECTS",
        "Method of grading",
        "Only after successfull completion of module(s)",
        "duration",
        "module level",
        "other prerequisites",
        "contents",
        "intended learning outcomes",
        "courses",
        "method of assessment",
        "Allocation of places",
        "Additional information",
        "Workload",
        "Teaching cycle",
        "Referred to in LPO I",
        "page",
    ]
    module_handbook_df = pd.DataFrame(columns=module_handbook_columns)



In [ ]:
def process_module_handbook(file_path, df):
    text = pdf_to_text(file_path, n=9)
    print("Done with converting to string")

    text_no_footer_header = remove_header_footer(text)
    print("Done with removing header and footer")

    tables = re.split(r"(?=Module title)", text_no_footer_header)[1:]
    print("Done with splitting into tables")


    
    for table in tables:
        values = extract_table_values(table)
        print("Done with extracting values for table")
        
        # Store the values in the matching column of the module_handbook_df
        row = {}
        for column in df.columns:
            if column in values:
                row[column] = values[column]
            else:
                row[column] = ""
        df = df.append(row, ignore_index=True)
    
    return df

In [ ]:
# apply the function to the module_handbook_df

module_handbook_columns = [
    "Module title",
    "Abbreviation",
    "Module coordinator",
    "Module offered by",
    "ECTS",
    "Method of grading",
    "Only after successfull completion of module(s)",
    "duration",
    "module level",
    "other prerequisites",
    "contents",
    "intended learning outcomes",
    "courses",
    "method of assessment",
    "Allocation of places",
    "Additional information",
    "Workload",
    "Teaching cycle",
    "Referred to in LPO I", "page"]
module_handbook_df = pd.DataFrame(columns=module_handbook_columns)
module_handbook_df.to_csv(
    "module_handbooks/master_information_systems.csv", index=False
)


module_handbook_df = process_module_handbook("module_handbooks/master_information_systems.pdf", module_handbook_df)
